## Check GPU

In [ ]:
!git clone https://github.com/dvschultz/AttnGAN

## Install pre-trained model and data

In [ ]:
# attn-gan bird model does not work , only coco
!gdown --id 1lqNG75suOuR_8gjoEPYNp8VyT_ufPPig

In [ ]:
# download all data for attn-gan coco model

#download pre-trained model
!gdown --id 1i9Xkg9nU74RAvkcqKE-rJYhjvzKAMnCi -O /content/AttnGAN/models/coco_AttnGAN2.pth

#download test/train captions
!gdown --id 1rSnbIGNDGZeHlsUlLdahj0RJ9oo6lgH9 -O /content/AttnGAN/data/coco.zip

%cd /content/AttnGAN/data/
!unzip coco.zip
!rm coco.zip
!rm -rf /content/AttnGAN/data/__MACOSX
%cd coco
!unzip /content/AttnGAN/data/coco/val2014-text.zip
!mv /content/AttnGAN/data/coco/val2014 /content/AttnGAN/data/coco/text

#download encoder
!gdown --id 1zIrXCE9F6yfbEJIbNP5-YrEe2pZcPSGJ -O /content/AttnGAN/DAMSMencoders/coco.zip
%cd /content/AttnGAN/DAMSMencoders
!unzip coco.zip
!rm coco.zip

#download font !wget https://github.com/opensourcedesign/fonts/blob/master/gnu-freefont_freemono/FreeMono.ttf -P /content/
!wget https://github.com/opensourcedesign/fonts/blob/master/gnu-freefont_freemono/FreeMono.ttf?raw=true -P /content/
!mv /content/FreeMono.ttf?raw=true /content/FreeMono.ttf

#go home
%cd /content/AttnGAN/code

In [ ]:
# Below this line write all of the captions. Put each caption on a new line.
%%writefile /content/AttnGAN/data/coco/example_filenames.txt
example_captions

## Create captions
Now we can write some captions. Add each caption on a new line below

In [ ]:
# Below this line write all of the captions. Put each caption on a new line.
%%writefile /content/AttnGAN/data/coco/example_captions.txt
A man riding a bicycle on the street.
A group of people sitting at a dining table.
A dog lying on a couch in a living room.
a photo of a homemade swirly pasta with broccoli carrots and onions.
a photo of a pizza with broccoli and onions.
A plate of pasta with vegetables on a wooden table.
A bowl of soup with bread on the side.
A close-up of a hamburger with lettuce and tomato.
A slice of pizza on a white plate.
A table full of various dishes and drinks at a restaurant.
a photo of a Turkish tea glass on a wooden table.
a street view from Istanbul with historical buildings.
a Turkish breakfast with cheese, olives, and tomatoes.
a photo of a Turkish tea glass on a wooden table.
a traditional Turkish breakfast with simit, cheese, olives, and tomatoes.
a mosque in Istanbul under a cloudy sky.
a street view in Istanbul with a tram and historical buildings.
a delicious Turkish kebab served with rice and salad.
a close-up of a Turkish doner kebab in pita bread.
a colorful traditional Anatolian kilim carpet on the floor.
a Turkish bazaar full of spices, lamps, and carpets.

In [ ]:
fixed_config = """
from __future__ import division
from __future__ import print_function

import os.path as osp
import numpy as np
from easydict import EasyDict as edict

__C = edict()
cfg = __C

# Dataset name: flowers, birds
__C.DATASET_NAME = 'birds'
__C.CONFIG_NAME = ''
__C.DATA_DIR = ''
__C.GPU_ID = 0
__C.CUDA = True
__C.WORKERS = 6

__C.RNN_TYPE = 'LSTM'   # 'GRU'
__C.B_VALIDATION = False

__C.TREE = edict()
__C.TREE.BRANCH_NUM = 3
__C.TREE.BASE_SIZE = 64

# Training options
__C.TRAIN = edict()
__C.TRAIN.BATCH_SIZE = 64
__C.TRAIN.MAX_EPOCH = 600
__C.TRAIN.SNAPSHOT_INTERVAL = 2000
__C.TRAIN.DISCRIMINATOR_LR = 2e-4
__C.TRAIN.GENERATOR_LR = 2e-4
__C.TRAIN.ENCODER_LR = 2e-4
__C.TRAIN.RNN_GRAD_CLIP = 0.25
__C.TRAIN.FLAG = True
__C.TRAIN.NET_E = ''
__C.TRAIN.NET_G = ''
__C.TRAIN.B_NET_D = True

__C.TRAIN.SMOOTH = edict()
__C.TRAIN.SMOOTH.GAMMA1 = 5.0
__C.TRAIN.SMOOTH.GAMMA3 = 10.0
__C.TRAIN.SMOOTH.GAMMA2 = 5.0
__C.TRAIN.SMOOTH.LAMBDA = 1.0

# Modal options
__C.GAN = edict()
__C.GAN.DF_DIM = 64
__C.GAN.GF_DIM = 128
__C.GAN.Z_DIM = 100
__C.GAN.CONDITION_DIM = 100
__C.GAN.R_NUM = 2
__C.GAN.B_ATTENTION = True
__C.GAN.B_DCGAN = False

__C.TEXT = edict()
__C.TEXT.CAPTIONS_PER_IMAGE = 10
__C.TEXT.EMBEDDING_DIM = 256
__C.TEXT.WORDS_NUM = 18

def _merge_a_into_b(a, b):
    if type(a) is not edict:
        return
    for k, v in a.items():
        if not k in b:
            raise KeyError('{} is not a valid config key'.format(k))
        old_type = type(b[k])
        if old_type is not type(v):
            if isinstance(b[k], np.ndarray):
                v = np.array(v, dtype=b[k].dtype)
            else:
                raise ValueError(('Type mismatch ({} vs. {}) '
                                  'for config key: {}').format(type(b[k]), type(v), k))
        if type(v) is edict:
            try:
                _merge_a_into_b(a[k], b[k])
            except:
                print('Error under config key: {}'.format(k))
                raise
        else:
            b[k] = v

def cfg_from_file(filename):
    import yaml
    from yaml import FullLoader
    with open(filename, 'r') as f:
        yaml_cfg = edict(yaml.load(f, Loader=FullLoader))
    _merge_a_into_b(yaml_cfg, __C)
"""

with open("/content/AttnGAN/code/miscc/config.py", "w") as f:
    f.write(fixed_config.strip())


In [ ]:
main_path = "/content/AttnGAN/code/main.py"

with open(main_path, "r") as f:
    lines = f.readlines()

with open(main_path, "w") as f:
    for line in lines:
        if "transforms.Scale(" in line:
            f.write(line.replace("transforms.Scale", "transforms.Resize"))
        else:
            f.write(line)


## Run

In [ ]:
!python /content/AttnGAN/code/main.py --cfg /content/AttnGAN/code/cfg/eval_coco.yml --gpu 0

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt


img_path = "/content/AttnGAN/models/coco_AttnGAN2/example_captions/0_a-photo-of-a-homemade-swirly-pasta-with-broccoli-carrots-and-onions._g2.png"


image = Image.open(img_path)
plt.imshow(image)
plt.axis('off')
plt.show()


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Klasör yolu
folder_path = "/content/AttnGAN/models/coco_AttnGAN2/example_captions"

# Görsel dosyalarının tam yollarını ve adlarını al
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_paths = [os.path.join(folder_path, f) for f in image_files]

# Görsel sayısı kadar alt alta subplot oluştur
fig, axs = plt.subplots(len(image_paths), 1, figsize=(8, len(image_paths) * 4))

# Tek bir resim varsa axs dizisi olmuyor, onu listeye çevir
if len(image_paths) == 1:
    axs = [axs]

# Her resmi ve başlığını subplot'a ekle
for ax, img_path, img_name in zip(axs, image_paths, image_files):
    image = Image.open(img_path)
    ax.imshow(image)
    ax.set_title(img_name, fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()
